# Using `hvPlot` and `Ibis` together

**Ibis**  
Ibis is a productivity API for working in Python and analyzing data in remote SQL-based data stores such as OmniSciDB. Inspired by the pandas toolkit for data analysis, Ibis provides a Pythonic API that compiles to SQL, and pushes the computational burden of the query to the server. In this way, users can query extremely large databases on remote servers without heavy local computation.
Ibis supports multiple SQL databases backends, and also supports pandas as a native backend. 


hvPlot can now take advantage of Ibis in order to display data stored on a database.  

This notebook demonstrates visualizations of an OmniSci Database data using hvPlot. (# to do: add sql lite database)

In order to test out some of the hvPlot functionality in Ibis, we'll need to connect to a Omnisci database. From there, we'll include a few ibis commands to manipulate the data in order to make some interesting plots.

In [ ]:
import ibis
import holoviews as hv
from holoviews import opts
import numpy as np
import hvplot.ibis
from matplotlib import cm

from ibis.backends import omniscidb as ibis_omniscidb

hv.extension('bokeh')

### 1. Establish the connection to the database

In [ ]:
client = ibis_omniscidb.connect(
    user="mapd", 
    password="HyperInteractive", 
    host="metis.mapd.com", 
    database="mapd", 
    port=443, 
    protocol='https'
)

### 2. Get the table

We are going to look at voting information from the 2012 presidential election (Obama vs. Romney).  The following saves the table we want in a variable called `votes`.  Displaying `votes` will show the metadata that ibis stores lazily in a task graph for now.

In [ ]:
votes = client.table('demo_vote_clean')
votes

Running the `.execute()` command will actually run the commands it is building in its task graph.


In [ ]:
votes.head().execute()

Looking at the different columns of the table we're considering, we can decide on some plots that may be interesting.  Because this includes election information, we have 2 distinct groups (Democrats and Republicans) that will make for some interesting plots.

Let's use `hvplot` to create histograms of votes for each party within each state.  In order to do this, we'll first need to convert the percentage of votes for each candidate into a concrete number of votes.  We'll use `Ibis` to do that.

### 3. Convert percentages to total votes

The following cell creates 2 new columns (`Democrat` and `Republican`) that represents the number of votes in each category.

In [ ]:
votes = votes.mutate(
        democrat=votes.Population.mul(votes.Obama),
        republican=votes.Population.mul(votes.Romney),
    )

### 4. Aggregate votes by state (StateAbb)

After computing our actual number of votes, we need to group the results by state for our histogram.

In [ ]:
voted = votes.groupby('StateAbb').aggregate(democrat=votes.democrat.sum(), republican=votes.republican.sum())

### 5. Overlayed `hvPlot`

Now, we have our data ready to create a histogram using `hvPlot`.  We can view these results by overlaying each group into one plot using the `*` operator as seen here:

In [ ]:
(
    voted.hvplot.bar('StateAbb', 'democrat')
    * voted.hvplot.bar('StateAbb', 'republican')
).opts(width=1000)

### 6. Side-by-Side plot

Alternatively, you can plot each histogram separately and view them side by side with the `+`, like so:

In [ ]:
voted = voted.head(10)
(
    (voted.hvplot.bar('StateAbb', 'democrat'))
    + voted.hvplot.bar('StateAbb', 'republican')
)

Because this data has coordinate information, it may be interesting to view the locations of all the data on a scatter plot, using `hvplot.scatter`.  With there is so much data, we make it easier to view the plot by setting `datashade=True` .

In [ ]:
votes.hvplot.scatter(x='Longitude', y='Latitude', datashade=True)

This plot shows us the locations of each zip code included in the database, but what will it look like if we separate Democrat and Replubican points into blue and red?  We'll need to do a little more manipulating with `Ibis` to prepare for plotting.

First, we'll create a new column called `color` which will label where the majority went to Democrats as `blue` and where the majority went to Republicans as `red`.

In [ ]:
votes = votes.mutate(
        color=ibis.case()
        .when(votes.democrat > votes.republican, 'blue')
        .else_('red')
        .end()

    )

Next, we'll create 2 new objects, `votesr` and `votesd` to contain just the information for Republicans and Democrats, respectively.  We'll do this by looking for the `red` or `blue` flags.

In [ ]:
votesr = votes[votes.color == 'red']

In [ ]:
votesd = votes[votes.color == 'blue']

Finally, we will plot each of these 2 groups, `votesr` and `votesd` in 2 separate images, using the `datashade` option.  Because
datashader using blues as its default color map, we don't need to change the Democrat group coloring, but for the Republican group, we will specify `cmap=cm.Reds`.  

In [ ]:
imr = votesr.hvplot.scatter(x='Longitude', y='Latitude', color='r', datashade=True, cmap=cm.Reds)
imd = votesd.hvplot.scatter(x='Longitude', y='Latitude', color='r', datashade=True)

As seen before, we use the `*` operator to plot them on top of each other.

In [ ]:
imr*imd